![code](../misc/curso.png)

# Generics in Scala

![a](../misc/genericeverywhere.jpg)

Generics is a widely spread feature of many programming languages, and Scala is no exception. We can find it virtually in every Scala library we want to use: 

![lists](../misc/scala-collections-list.png)

![list-map](../misc/scala-api-list-map.png)

![Scala Collections](../misc/scala-api-list-prepended.png)

![spark-dataset](../misc/spark-api-dataset.png)

![dataset-flatmap](../misc/spark-api-dataset-flatmap.png)

![spark-groupbykey](../misc/spark-api-dataset-groupbykey.png)

![akka-flow](../misc/akka-api-flow.png)

![akka-flow-fold](../misc/akka-api-flow-foldasync.png)

![akka-flow-runwith](../misc/akka-api-flow-runwith.png)

# Generic features have a long history

The first programming language to implement them was ML (1975). Then, they come Haskell, C++, ... and, of course, Java: 

![java-generics](../misc/genericsjava.png)

![odersky](../misc/odersky.png)

![lambdaman](../misc/lambdaman.png)
Prof. Philip Wadler @LambdaWorld'16

# Generics, what and why?

Let's motivate the need for generics with a simple, related problem, which is not solved with generics, but it will serve as an inspiration anyway. We start from this definition of lists of integers: 

In [ ]:
sealed abstract class IntList
case class End() extends IntList
case class Cons(head: Int, tail: IntList) extends IntList

and a function to add the number five to the end of the list: 

In [ ]:
def addFive(list: IntList): IntList = 
    list match {
        case End() => Cons(5, End())
        case Cons(head, tail) => Cons(head, addFive(tail))
    }

There is something wrong with this function: it's too specific. What if we want to add a different number, say three? We may copy & paste the above definition and replace five with three: 

In [ ]:
def addThree(list: IntList): IntList = 
    list match {
        case End() => Cons(3, End())
        case Cons(head, tail) => Cons(head, addThree(tail))
    }

But, of course, we may rather abstract the function from the number to be inserted, and extends the signature with a new parameter. 

In [ ]:
def addNumber(n: Int, list: IntList): IntList = 
    list match {
        case End() => Cons(n, End())
        case Cons(head, tail) => Cons(head, addNumber(n, tail))
    }

Indeed, that's the very purpose of functions: to obtain more modular programs.

In [ ]:
def addFive(list: IntList): IntList = addNumber(5, list)
def addThree(list: IntList): IntList = addNumber(3, list)

But, did we finish here our modularization work? Not yet. Let's say that we now want a list of strings. What do we do, copy & paste again?

In [ ]:
sealed abstract class StringList
case class End() extends StringList
case class Cons(head: String, tail: StringList) extends StringList

and the same for the function `addString`?

In [ ]:
def addString(n: String, list: StringList): StringList = 
    list match {
        case End() => Cons(n, End())
        case Cons(head, tail) => Cons(head, addString(n, tail))
    }

Couldn't we do something similar to what we did before? Yes, but now we have to abstract the definition of `addString` and `StringList` from the *type* `String`. This is how we get to our first generic declaration:

In [ ]:
sealed abstract class List[A]
case class End[A]() extends List[A]
case class Cons[A](head: A, tail: List[A]) extends List[A]

and we can do the same for the generic method definition:

In [ ]:
def add[A](n: A, list: List[A]): List[A] = 
    list match {
        case End() => Cons(n, End())
        case Cons(head, tail) => Cons(head, add[A](n, tail))
    }

Now, the `addNumber` and `addString` functions can be fully modularised as follows:

In [ ]:
def addNumber(n: Int, l: List[Int]): List[Int] = 
    add[Int](n, l)
def addString(n: String, l: List[String]): List[String] = 
    add[String](n, l)

The important thing to remember is that we can treat value parameters and type parameters as just that, i.e. **parameters**. And when we invoke a generic method, we will normally pass values, but also types. There is a difference concerning types, however: types can be normally inferred by the Scala compiler, whereas values can not (except if they are declared implicitly, as we will see in the next session). Thus, we can invoke our generic function as follows:

In [ ]:
add[Int](5, Cons(5, Cons(4, End())))

but also in the following way: 

In [ ]:
add(5, Cons(5, Cons(4, End())))

# The power of  _parametric polymorphism_

The kind of generic programming illustrated above is called *parametric polymorphism*. It constrasts with other kinds of polymorphism such as *ad-hoc polymorphism* (which will be mentioned in the session about implicits), and *subtype polymorphism* (which will be exemplified later on in this session). 

In order to better undertand this kind of polymorphism we will consider a number of examples. For instance, let's consider the following monomorphic function. How many functions with this signature are there?

In [ ]:
def foo(i: Int): Int = ???

a lot, actually: ${(2^{32})}^{2^{32}}$

In contrast, how many functions are there with the following polymorphic signature?

In [ ]:
def foo[A](a: A): A = ???

Well, we may pretend to give different implementations doing horrible things, as for instance: 

In [ ]:
def foo[A](a: A): A = 
    1.asInstanceOf[A]

This function compiles, but it won't always work at runtime: 

In [ ]:
foo[Int](1)

In [ ]:
foo[String]("")

Here it is another implementation, this time taking advantage that `Nothing` (the "type" of exceptions) is a subtype of every other type:

In [ ]:
def foo[A](a: A): A = throw new RuntimeException("error")

We may even try (and achieve) to dispatch a different behaviour depending on the type:

In [ ]:
def foo[A](a: A): A = 
    if (a.isInstanceOf[String]) ???
    else if (a.isInstanceOf[Int]) ??? 
    else ???

or the particular value: 

In [ ]:
def foo[A](a: A): A = 
    if (a.toString == "") ??? 
    else if (a.hashCode == 0) ??? 
    else if (a.equals(1.0)) ???
    else ???

But, actually, in logical justice, just from the meaning of the signature, we can only do one thing: 

In [ ]:
def foo[A](a: A): A = a

![knownothing](../misc/knownothing.jpg)

This is a most relevant feature of parametric polymorphism: we don't know anything about generic parameters. This sole fact retricts our implementations a lot, and, accordingly, makes the corresponding signatures very expressive. 

Another example. There are many different implementations of this function:  

In [ ]:
def apply(f: Int => String, i: Int): String = 
    ???

But only one for its generic version:

In [ ]:
def apply[A, B](f: A => B, a: A): B = 
    f(a)

Yet another example:

In [ ]:
def andThen(f1: Int => String, f2: String => Boolean): Int => Boolean = 
    (i: Int) => ??? : Boolean

What about its generic version?

In [ ]:
def andThen[A, B, C](f1: A => B, f2: B => C): A => C = ???

This is only a little bit more complicated, but let's ask lambda man for help!

![lambdamanhelp](../misc/lambdamanhelp.png)

In [ ]:
def andThen[A, B, C](f1: A => B, f2: B => C): A => C = 
    (a: A) => f2(f1(a : A) : B) : C

Of course, It's not always the case that there is only one possible implementation. In the following case, we may rearrange the list values in multiple ways: 

In [ ]:
def foo[A](list: List[A]): List[A] = 
    ???

But we can be sure that any implementation of `foo` must satisfy the following property: 

![freetheorems](../misc/lambdamanfreetheorems.png)

In [ ]:
def map[A, B](f: A => B): List[A] => List[B] = ???

In [ ]:
def property[A, B](f: A => B, x: List[A]): Boolean = 
    ((foo[A](_)) andThen map[A, B](f))(x) == 
    (map[A, B](f) andThen (foo[B](_)))(x)

# Generics and subtyping

Let's turn now to explain the interactions between generics and subtyping. We will use the following inheritance hierarchy:

In [ ]:
class Complex(val real: Double, val img: Double)
class Real(val value: Double)              extends Complex(value, 0.0)
class Rational(val num: Int, val den: Int) extends Real(num/den)
class IntegerN(val int: Int)               extends Rational(int,1)
class Natural(val nat: Int)                extends IntegerN(nat.abs)

In [ ]:
val `1+2i`: Complex = new Complex(1.0,2.0)
val π: Real = new Real(3.1415)
val `2/3`: Rational = new Rational(2, 3)
val `-1`: IntegerN = new IntegerN(-1)
val `1`: Natural = new Natural(1)

According to this hierarchy, naturals are integers, integers are rational numbers, and so on. This means that we can use a natural number whenever we need an integer, e.g. in passing a value to a function, initialising a variable, etc. Thus, given function `foo` expecting a `Rational` number: 

In [ ]:
def foo(c: Rational): Rational = c

we may safely pass a rational, integer or natural number: 

In [ ]:
foo(`2/3`)
foo(`-1`)
foo(`1`)

but not a real or complex number: 

In [ ]:
// foo(`1+2i`)
// foo(π)

### Covariant and contravariant parameters

Let's now investigate which subtyping relationships hold between functions. For instance, given the following higher-order function:

In [ ]:
def foo(f: Rational => Rational): Rational = 
    f(new Rational(4,2))

we can, of course, pass this function:

In [ ]:
val times2: Rational => Rational = 
    r => new Rational(r.num*2,r.den)

In [ ]:
foo(times2)

But, can we pass this function?

In [ ]:
val idiv: Rational => IntegerN = 
    r => new IntegerN(r.num/r.den)

In [ ]:
foo(idiv)

And what about this one?

In [ ]:
val rr2: Real => Rational = 
    _ => new Rational(0,1)

In [ ]:
foo(rr2)

This makes sense. In the former case, we pass a function which returns a value of a more specific type, `Integer`, which is perfectly compatible with the demanded type, `Rational`. In the latter one, we pass a function that is able to be applied to `Real` values, and, in particular, to `Rational`s, which is what will happen. So, no problem at all.

What about this function?

In [ ]:
val rr3: Natural => Rational = _ => new Rational(0,1)

In [ ]:
//foo(rr3)

And for the final case:

In [ ]:
val rr4: Rational => Complex = _ => new Complex(0.0, 0.0)

In [ ]:
// foo(rr4)

Neither case work, and it make sense. In the former one, we pass a function that can only receive naturals, but in the body of `apply` we see that it will be applied to a rational number. In the second case, `apply` is given a function that returns a complex value, whereas it can only handle rationals.

So, these are the subtype relationships which are desired:

![function1generics](../misc/function1generics.png)

Let's see now how Scala defines `Function1` so that the previous subtyping relationships hold. We may start by abstracting the following monomorphic function from the input and output types:

In [ ]:
trait Function1IntToString{
    def apply(i: Int): String
}

obtaining the following definition:

In [ ]:
trait Function1[I, O]{
    def apply(i: I): O
}

This is a good start. Let's redefine our `foo` function and attempt to reproduce the desired behaviour.

In [ ]:
def foo(f: Function1[Rational, Rational]): Rational = 
    f(`2/3`)

Of course, it work for the obvious case:

In [ ]:
val times2: Function1[Rational, Rational] = 
    new Function1[Rational, Rational]{
        def apply(r: Rational): Rational = 
            new Rational(r.num*2,r.den)
    }

In [ ]:
foo(times2)

But it fails when the return type is a subtype:

In [ ]:
val idiv: Function1[Rational, IntegerN] = 
    new Function1[Rational, IntegerN]{
        def apply(r: Rational): IntegerN = 
            new IntegerN(r.num/r.den)
    }

In [ ]:
// foo(idiv)

The compiler error gives us a definitive clue: we have to declare the second parameter type of `Function1` *covariantly* :

In [ ]:
trait Function1[I, +O]{
    def apply(i: I): O
}

def foo(f: Function1[Rational, Rational]): Rational = 
    f(`2/3`)

val idiv: Function1[Rational, IntegerN] = 
    new Function1[Rational, IntegerN]{
        def apply(r: Rational): IntegerN = 
            new IntegerN(r.num/r.den)
    }

In [ ]:
foo(idiv)

We still have a problem, however: 

In [ ]:
val rr2 = new Function1[Real, Rational]{
    def apply(r: Real): Rational = 
        new Rational(0,1)
}

In [ ]:
// foo(rr2)

In this case we need to make the first type parameter *contravariant* : 

In [ ]:
trait Function1[-I, +O]{
    def apply(i: I): O
}

def foo(f: Function1[Rational, Rational]): Rational = 
    f(`2/3`)

val idiv: Function1[Rational, IntegerN] = 
    new Function1[Rational, IntegerN]{
        def apply(r: Rational): IntegerN = 
            new IntegerN(r.num/r.den)
    }

val rr2 = new Function1[Real, Rational]{
    def apply(r: Real): Rational = 
        new Rational(0,1)
}

And all the cases work as expected:

In [ ]:
foo(idiv)
foo(rr2)

### Subtype polymorphism

Given the following generic definition of lists:

In [ ]:
sealed abstract class MyList[A]
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

We can create a list of rationals with rational numbers, but not with reals or complex numbers: 

In [ ]:
val a: MyList[Rational] = 
    Cons(`2/3`, Cons(`-1`, Cons(`1`, End())))

Similar question than before: which kinds of lists would be safe to pass to this function?

In [ ]:
def `prepend_2/3`(l: MyList[Rational]): MyList[Rational] = 
    Cons(`2/3`, l)

We may of course pass a list of rationals: 

In [ ]:
val ratList: MyList[Rational] = Cons(`1`, Cons(`-1`, End()))
`prepend_2/3`(ratList)

But we can't pass a list of integers, which it's a pity: 

In [ ]:
val intList: MyList[IntegerN] = Cons(`1`, Cons(`-1`, End()))
// `prepend_2/3`(intList)

The very same compiler gives us a clue to solve the problem: "You may wish to define A as +A instead." In sum, we want the following relationship: 

![listgenerics](../misc/listgenerics.png)

Let's make that change: 

In [ ]:
sealed abstract class MyList[+A]
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

def `prepend_2/3`(l: MyList[Rational]): MyList[Rational] = 
    Cons(`2/3`, l)

val intList: MyList[IntegerN] = Cons(`1`, Cons(`-1`, End()))

and now it works:

In [ ]:
`prepend_2/3`(intList)

How can we generalise the function `prepend_2/3`?  This should be ok:

In [ ]:
def prepend[N](l: MyList[N], r: N): MyList[N] = 
    Cons(r, l)

Indeed, it supports those cases in which the actual types of `r` and `l` are different but compatible. For instance:

In [ ]:
prepend(intList, π) // since MyList[Int] <: MyList[Real]
prepend(intList, `1`: Natural) // since Natural <: Int

If we intend, however, to implement this function as a method class, we run into trouble:

In [ ]:
sealed abstract class MyList[+A]{
    //def prepend(a: A): MyList[A] = 
      //  Cons(a, this)
}
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

The problem is that, now, the type of the list is fixed in the method invocation. This implies that if we want to allow for the resulting list type to be more generic, we have to twist the method declaration a little bit with a so-called *supertype bound*:

In [ ]:
sealed abstract class MyList[+A]{
    def prepend[B >: A](a: B): MyList[B] = 
        Cons(a, this)
}
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

val intList: MyList[IntegerN] = Cons(`1`, Cons(`-1`, End()))

And now everything works:

In [ ]:
intList.prepend[IntegerN](`-1`)
intList.prepend[Real](π)
intList.prepend(`1`)
// but: intList.prepend[Natural](`1`)

The `List` method `prepend` is an example of subtype polymorphism, a kind of polymorphism which is not universal but restricted to types for which a certain subtyping relationship hold. 

Last, similarly to super-type bounds, we may also use subtype bounds, as in the following declaration, where we only allow the method to work for `Rational`s:

In [ ]:
def prependS[N <: Rational](l: MyList[N], r: N): MyList[N] = 
    Cons(r, l)

Now, if we pass a list of integers it will work in the same way as in the previous definition of `prepend`: 

In [ ]:
prependS(intList,`-1`)

but it will fail with lists of reals:

In [ ]:
// prependS(Cons(π, End()), π)

But, which is the difference of `prependS` with the following monomorphic function?

In [ ]:
def prependR(l: MyList[Rational], r: Rational): MyList[Rational] = 
    Cons(r, l)

Certainly, this function will allow us to pass the same types of values:

In [ ]:
prependR(intList: MyList[IntegerN], `1`: Natural)

but note that the resulting type is too general, compared to the one obtained here:

In [ ]:
prependS(intList: MyList[IntegerN], `1`: Natural)

# Futher topics

There are many topics related to generics we may talk about (for some hours more :):
* Higher-kinded generics. So-called type constructor parameters. 
* The curry-howard isomorphism. Showing how generic signatures correspond to propositions of second-order logic, and their implementations to proofs.
* Parametric definitions of ADTs. How we can give isomorphic definitions of tuples and eithers without data types at all.
* Generics in dotty. Improvements over the Scala compiler related to generics. 

# Conclusion

Some takeaways: 
* Generic parameters are additional parameters of types and functions, even if you don't see them in method invocations or instance declarations!
* Use them whenever you want your code to be more modular and more reusable. 
* Take generic signatures seriously, and don't cheat (even if the Scala compiler allows you to be impure)!
* Variance and contravariance annotations are difficult to reason about. Try to avoid them ... by not using inheritance!